In [2]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns

In [30]:
cd '/Users/admin/Jedha/'

'/Users/admin/Jedha'

In [38]:
from s3_credentials import *

In [40]:
import boto3
session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")

#for bucket in s3.buckets.all():
#        print(bucket.name)

In [ ]:
# real
obj = s3.Object('jedha-fake-reviews-project', "datasets/real_reviews_raw.csv")
real = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# fake
obj = s3.Object('jedha-fake-reviews-project', "datasets/fake_reviews_raw.csv")
fake = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)




In [ ]:
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

pivot_table = real.loc[:, ['url','restaurant_average_rating','restaurant_reviews_count','restaurant_expensiveness','restaurant_name']].drop_duplicates('url')

fake_merged = (pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left'))
fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered]

full_dataset = pd.concat([real, fake_dataset_reworked])
full_dataset = full_dataset.reset_index(drop = True)
full_dataset = full_dataset.drop_duplicates()
full_dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))

In [8]:
lang = []
for i in range(len(dataset)):
    item = dataset['review_lite'].iloc[i]
    l = get_lang(item)
    lang.append(l)
    if i % 500 == 0:
        print(i, 'ok')

0 ok
500 ok
1000 ok
1500 ok
2000 ok
2500 ok
3000 ok
3500 ok
4000 ok
4500 ok
5000 ok
5500 ok
6000 ok
6500 ok
7000 ok
7500 ok
8000 ok
8500 ok
9000 ok
9500 ok
10000 ok
10500 ok
11000 ok
11500 ok
12000 ok
12500 ok
13000 ok
13500 ok
14000 ok
14500 ok
15000 ok
15500 ok
16000 ok
16500 ok
17000 ok
17500 ok
18000 ok
18500 ok
19000 ok
19500 ok
20000 ok
20500 ok
21000 ok
21500 ok
22000 ok
22500 ok
23000 ok
23500 ok
24000 ok
24500 ok
25000 ok
25500 ok
26000 ok
26500 ok
27000 ok
27500 ok
28000 ok
28500 ok
29000 ok
29500 ok
30000 ok
30500 ok
31000 ok
31500 ok
32000 ok
32500 ok
33000 ok
33500 ok
34000 ok
34500 ok
35000 ok
35500 ok
36000 ok
36500 ok
37000 ok
37500 ok
38000 ok
38500 ok
39000 ok
39500 ok
40000 ok
40500 ok
41000 ok
41500 ok
42000 ok
42500 ok
43000 ok
43500 ok
44000 ok
44500 ok
45000 ok
45500 ok
46000 ok
46500 ok
47000 ok
47500 ok
48000 ok
48500 ok
49000 ok
49500 ok
50000 ok
50500 ok
51000 ok
51500 ok
52000 ok
52500 ok
53000 ok
53500 ok
54000 ok
54500 ok
55000 ok
55500 ok
56000 ok
56500 o

In [7]:

dataset['lang'] = dataset['text_reviews'].apply(lambda x: detect(x))

In [76]:
dataset['text_review']

'https://www.yelp.fr/not_recommended_reviews/le-buisson-d-argent-paris-3'

In [3]:
def get_lang(x):

    try:
        lang = detect(str(x))
    except:
        lang = 'not_found'

    return lang

In [4]:
path_to_dataset = '/Users/admin/Jedha/Fake_reviews_detection/Scraping/Yelp_Paris_Scraping_reviews/dataset/full_dataset_reworked.csv'

In [5]:
dataset = pd.read_csv(path_to_dataset, index_col = 0)

In [6]:
dataset['text_length'] = dataset['text_review'].apply(lambda x : len(x))

In [31]:
dataset['id'] = dataset['username'] + str(dataset['user_friends_count']) + str(dataset['user_location']) + str(dataset['user_reviews_count']) + str(dataset['user_total_image_posted'])

In [19]:
print('real_review | mean_user_review_count: ', dataset.loc[dataset['is_real_review']==1]['user_reviews_count'].mean())
print('fake_review | mean_user_review_count: ', dataset.loc[dataset['is_real_review']==0]['user_reviews_count'].mean())
print('-----------------------------------------------------------------------------------------')
print('real_review | mean_user_friend_count: ', dataset.loc[dataset['is_real_review']==1]['user_friends_count'].mean())
print('fake_review | mean_user_friend__count: ', dataset.loc[dataset['is_real_review']==0]['user_friends_count'].mean())
print('-----------------------------------------------------------------------------------------')
print('real_review | mean_user_image_posted_count: ', dataset.loc[dataset['is_real_review']==1]['user_total_image_posted'].mean())
print('fake_review | mean_user_image_posted_count: ', dataset.loc[dataset['is_real_review']==0]['user_total_image_posted'].mean())


real_review | mean_user_review_count:  95.22160705003445
fake_review | mean_user_review_count:  19.7975533181769
-----------------------------------------------------------------------------------------
real_review | mean_user_friend_count:  43.63132381562564
fake_review | mean_user_friend__count:  240.25062249648155
-----------------------------------------------------------------------------------------
real_review | mean_user_image_posted_count:  161.56677103103127
fake_review | mean_user_image_posted_count:  58.29793340987371


19.7975533181769